<h2 align = "center">Sprawozdanie mofit</h2>
<h4 align="center">Łukasz Puchała, Karol Grzywa</h4>

In [12]:
import numpy as np
from numba import njit
import math
import matplotlib.pyplot as plt
from math import sin, cos, pi, e, exp
from scipy.linalg import eigh
import Utilis

m = 0.067 # m0
h = 1
at2meV = 27211.6
ab = 0.05292
j = 0 + 1j
at2s = 2.42e-5*1e-12

L = 100/ab
N = 2
a = L/(2*N)

In [13]:
grid_N2_L100 = Utilis.ReadFile('Files/wezly_N_2_L_100.dat')
wezly = np.array(grid_N2_L100)

In [14]:
nlg_N2_l100 = Utilis.ReadFile('Files/nlg_N_2_L_100.dat.txt')
nlg = np.array(nlg_N2_l100)

In [15]:
# funkcja, co bierze poz. wezla naroznego elementu k o loklanym nr i
print( Utilis.xy_nlg(2, 5, nlg, wezly) )
# wspóółrzędne rzeczywiste punktu 
print( Utilis.xryr(k= 1, ksi1= 1, ksi2= -1, nlg= nlg, wezly= wezly) )

(-12.5, -50.0)
(-25.0, -50.0)
